In [ ]:
!pip install jsonlines

###**Load data from Google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
drive_path = '/content/drive/MyDrive/Fine_tuned_BERT'

Mounted at /content/drive


###**Main code**

###**Imports**

In [ ]:
import pandas as pd
import jsonlines
import json
import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
# Function to read JSONL file into DataFrame
def read_jsonl(file_path):
    data = []
    with jsonlines.open(file_path, 'r') as reader:
        for line in reader:
            data.append(line)
    return pd.DataFrame(data)

train_path = drive_path+"/train.jsonl"
test_path=drive_path+"/test.jsonl"
dev_path=drive_path+"/dev.jsonl"
label_path=drive_path+"/secs.jsonl"

# Load JSONL data into DataFrame
train_data = read_jsonl(train_path)
test_data = read_jsonl(test_path)
dev_data = read_jsonl(dev_path)

#Get all labels
all_labels=[]

with open(label_path, "r") as f:
    for line in f:
        obj = json.loads(line)
        all_labels.append(obj.get("id"))

print(len(all_labels))

100


In [ ]:
import os
import h5py
import numpy as np

model_path = drive_path+"/fine_tuned_model/"
if not os.path.exists(model_path):
    model_path = "nlpaueb/legal-bert-small-uncased"

print(model_path)
# Preprocess data
def preprocess_texts(texts):
    return [' '.join(case) for case in texts]

train_texts, train_labels = preprocess_texts(train_data['text']), train_data['labels']
test_texts, test_labels = preprocess_texts(test_data['text']), test_data['labels']
dev_texts, dev_labels = preprocess_texts(dev_data['text']), dev_data['labels']

# Load LegalBERT model and tokenizer
model_name = "nlpaueb/legal-bert-small-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_path)

train_encodings = tokenizer(train_texts, padding=True, truncation=True, return_tensors='pt')
test_encodings = tokenizer(test_texts, padding=True, truncation=True, return_tensors='pt')
dev_encodings = tokenizer(dev_texts, padding=True, truncation=True, return_tensors='pt')


/content/drive/MyDrive/Fine_tuned_BERT/fine_tuned_model/


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/989 [00:00<?, ?B/s]

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(classes=all_labels)

train_labels_encoded = mlb.fit_transform(train_labels)
test_labels_encoded = mlb.fit_transform(test_labels)
dev_labels_encoded = mlb.fit_transform(dev_labels)
train_labels_tensor = torch.tensor(train_labels_encoded)
test_labels_tensor = torch.tensor(test_labels_encoded)
dev_labels_tensor = torch.tensor(dev_labels_encoded)

print(train_labels_tensor.shape)
print(test_labels_tensor.shape)
print(dev_labels_tensor.shape)

print(mlb.classes_)

torch.Size([42835, 100])
torch.Size([13039, 100])
torch.Size([10200, 100])
['Section 2 in The Indian Penal Code' 'Section 3 in The Indian Penal Code'
 'Section 4 in The Indian Penal Code' 'Section 5 in The Indian Penal Code'
 'Section 13 in The Indian Penal Code'
 'Section 34 in The Indian Penal Code'
 'Section 107 in The Indian Penal Code'
 'Section 109 in The Indian Penal Code'
 'Section 114 in The Indian Penal Code'
 'Section 120 in The Indian Penal Code'
 'Section 120B in The Indian Penal Code'
 'Section 143 in The Indian Penal Code'
 'Section 147 in The Indian Penal Code'
 'Section 148 in The Indian Penal Code'
 'Section 149 in The Indian Penal Code'
 'Section 155 in The Indian Penal Code'
 'Section 156 in The Indian Penal Code'
 'Section 161 in The Indian Penal Code'
 'Section 164 in The Indian Penal Code'
 'Section 173 in The Indian Penal Code'
 'Section 174A in The Indian Penal Code'
 'Section 186 in The Indian Penal Code'
 'Section 188 in The Indian Penal Code'
 'Section 190 i

In [ ]:
# Define PyTorch datasets
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels_tensor)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels_tensor)
dev_dataset = TensorDataset(dev_encodings['input_ids'], dev_encodings['attention_mask'], dev_labels_tensor)

del train_encodings
del test_encodings
del dev_encodings
del train_labels_tensor
del test_labels_tensor
del dev_labels_tensor


In [ ]:
# Define data loaders
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4)
dev_loader = DataLoader(dev_dataset, batch_size=4)


In [ ]:

# Modify model head for classification
num_classes = 100
model.config.num_labels = num_classes

print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 512, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, e

In [ ]:
model.classifier = torch.nn.Linear(in_features=model.classifier.in_features, out_features=100)
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 512, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, e

In [ ]:
device = torch.device('cuda')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 512, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, e

In [ ]:
import torch.nn as nn
from sklearn.metrics import precision_score, recall_score, f1_score

# Define the directory where you want to save the model
output_dir = drive_path + "/fine_tuned_model/"

# Create the directory if it doesn't exist
import os
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.BCEWithLogitsLoss()

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        logits = outputs.logits
        loss = criterion(logits, labels.float())  # Convert labels to float for BCELoss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Validation loop
    model.eval()
    all_true_labels = []
    all_predicted_labels = []

    with torch.no_grad():
        for batch in dev_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            # Convert logits to probabilities using sigmoid activation
            probabilities = torch.sigmoid(outputs.logits)

            # Round probabilities to obtain binary predictions
            predicted_labels = (probabilities > 0.5).float()

            all_true_labels.extend(labels.cpu().numpy())
            all_predicted_labels.extend(predicted_labels.cpu().numpy())

    # Calculate precision, recall, and F1-score using sklearn
    all_true_labels = np.array(all_true_labels)
    all_predicted_labels = np.array(all_predicted_labels)

    accuracy = (all_predicted_labels == all_true_labels).all(axis=1).mean()
    precision = precision_score(all_true_labels, all_predicted_labels, average='micro')
    recall = recall_score(all_true_labels, all_predicted_labels, average='micro')
    f1 = f1_score(all_true_labels, all_predicted_labels, average='micro')

    print(f'Epoch {epoch + 1}: Validation Accuracy: {accuracy}, Validation Precision: {precision}, Validation Recall: {recall}, Validation F1-score: {f1}')

    # Save the model and tokenizer
    model.save_pretrained(output_dir)
    print("Fine-tuned model saved to:", output_dir)


Epoch 1: Validation Accuracy: 0.13607843137254902, Validation Precision: 0.7966489982930555, Validation Recall: 0.444274656178762, Validation F1-score: 0.5704314822855857
Fine-tuned model saved to: /content/drive/MyDrive/Fine_tuned_BERT/fine_tuned_model/
Epoch 2: Validation Accuracy: 0.19549019607843138, Validation Precision: 0.7709870680207496, Validation Recall: 0.52869560860743, Validation F1-score: 0.6272569212250902
Fine-tuned model saved to: /content/drive/MyDrive/Fine_tuned_BERT/fine_tuned_model/
Epoch 3: Validation Accuracy: 0.2299019607843137, Validation Precision: 0.7606101694915254, Validation Recall: 0.5620882286630426, Validation F1-score: 0.6464512597415694
Fine-tuned model saved to: /content/drive/MyDrive/Fine_tuned_BERT/fine_tuned_model/
Epoch 4: Validation Accuracy: 0.24774509803921568, Validation Precision: 0.7766492218702657, Validation Recall: 0.5650692652621558, Validation F1-score: 0.654176877456027
Fine-tuned model saved to: /content/drive/MyDrive/Fine_tuned_BERT

In [ ]:
import torch.nn as nn
from sklearn.metrics import precision_score, recall_score, f1_score

# Test loop
model.eval()
all_true_labels = []
all_predicted_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        # Convert logits to probabilities using sigmoid activation
        probabilities = torch.sigmoid(outputs.logits)

        # Round probabilities to obtain binary predictions
        predicted_labels = (probabilities > 0.5).float()

        all_true_labels.extend(labels.cpu().numpy())
        all_predicted_labels.extend(predicted_labels.cpu().numpy())

# Calculate precision, recall, and F1-score using sklearn
all_true_labels = np.array(all_true_labels)
all_predicted_labels = np.array(all_predicted_labels)

accuracy = (all_predicted_labels == all_true_labels).all(axis=1).mean()
precision = precision_score(all_true_labels, all_predicted_labels, average='micro')
recall = recall_score(all_true_labels, all_predicted_labels, average='micro')
f1 = f1_score(all_true_labels, all_predicted_labels, average='micro')

print('Test Accuracy:', accuracy)
print('Test Precision:', precision)
print('Test Recall:', recall)
print('Test F1-score:', f1)


Test Accuracy: 0.2820001533859959
Test Precision: 0.7455990914253265
Test Recall: 0.5934171742975044
Test F1-score: 0.6608602409374897


In [ ]:
# Define the directory where you want to save the model
output_dir = drive_path+"/fine_tuned_model/"

# Create the directory if it doesn't exist
import os
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the model and tokenizer
model.save_pretrained(output_dir)

print("Fine-tuned model saved to:", output_dir)


Fine-tuned model saved to: /content/drive/MyDrive/Fine_tuned_BERT/fine_tuned_model/


In [ ]:

import torch.nn as nn
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate_model(model, dataloader):
    model.eval()
    all_true_labels = []
    all_predicted_labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            # Convert logits to probabilities using sigmoid activation
            probabilities = torch.sigmoid(outputs.logits)

            # Round probabilities to obtain binary predictions
            predicted_labels = (probabilities > 0.5).float()

            all_true_labels.extend(labels.cpu().numpy())
            all_predicted_labels.extend(predicted_labels.cpu().numpy())

    # Calculate precision, recall, and F1-score using sklearn
    all_true_labels = np.array(all_true_labels)
    all_predicted_labels = np.array(all_predicted_labels)

    accuracy = (all_predicted_labels == all_true_labels).all(axis=1).mean()
    precision = precision_score(all_true_labels, all_predicted_labels, average='micro')
    recall = recall_score(all_true_labels, all_predicted_labels, average='micro')
    f1 = f1_score(all_true_labels, all_predicted_labels, average='micro')

    return accuracy, precision, recall, f1

original_model = "nlpaueb/legal-bert-small-uncased"
original_model = BertForSequenceClassification.from_pretrained(original_model)
num_classes = 100
original_model.config.num_labels = num_classes
original_model.classifier = torch.nn.Linear(in_features=original_model.classifier.in_features, out_features=100)

device=torch.device('cuda')
original_model.to(device)

original_model_accuracy, original_model_precision, original_model_recall, original_model_f1_score = evaluate_model(original_model, test_loader)
print("Original Model Metrics:")
print(f"Accuracy: {original_model_accuracy}")
print(f"Precision: {original_model_precision}")
print(f"Recall: {original_model_recall}")
print(f"F1 Score: {original_model_f1_score}")


pytorch_model.bin:   0%|          | 0.00/141M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-small-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Original Model Metrics:
Accuracy: 0.0
Precision: 0.03912017561062014
Recall: 0.4860679897818825
F1 Score: 0.07241238996601278
